In [ ]:
%run main

In [ ]:
import main
import algo
import utils

In [1]:
import torch

In [3]:
q = torch.zeros(2)

In [4]:
q[0] += 1
q

tensor([1., 0.])

In [1]:
import main
import algo
import utils

/Users/xiejiahan/miniforge3/envs/RLinDR/lib/python3.8/site-packages/pyparsing.py:943: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  collections.MutableMapping.register(ParseResults)
/Users/xiejiahan/miniforge3/envs/RLinDR/lib/python3.8/site-packages/matplotlib/__init__.py:169: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
/Users/xiejiahan/miniforge3/envs/RLinDR/lib/python3.8/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


In [ ]:
os.chdir('/content/drive/MyDrive/MAAC1/maac')
%run main